In [ ]:
import sys
sys.path.append('../input/nubes-score')

from copy import deepcopy
from collections import defaultdict

eval_model = 1

if eval_model == 1:
    from ner_eval import collect_named_entities
    from ner_eval import compute_metrics
    from ner_eval import compute_precision_recall_wrapper
    from ner_eval import Evaluator
elif eval_model == 2:
    from ner_eval2 import collect_named_entities
    from ner_eval2 import compute_metrics
    from ner_eval2 import compute_precision_recall_wrapper
    from ner_eval2 import Evaluator
    
import numpy as np

In [ ]:
a2 = np.load('../input/nubes-score/arr.npy')

categories = ['NEG', 'NSCO', 'UNC', 'USCO']#,  '[CLS]', '[SEP]', '[PAD]']

In [ ]:
sentences = a2[0][0]
y_true    = a2[0][1]
y_pred    = a2[0][2]

In [ ]:
print(len(sentences), len(y_true), len(y_pred))

In [ ]:
#kata = []
#for sen, yt, yp in zip(sentences, y_true, y_pred):
#    for el_se, el_yt, el_yp in zip(sen, yt, yp):
#        kata.append([el_se, el_yt, el_yp])
#    kata.append(['--', '--', '--'])

In [ ]:
#np.savetxt('./myfile.csv', kata, delimiter=',', fmt='%s')

In [ ]:
metrics_results = {'correct': 0, 'incorrect': 0, 'partial': 0, 'missed': 0, 'spurious': 0, 'possible': 0, 'actual': 0, 'precision': 0, 'recall': 0}

# overall results
results = {'strict':   deepcopy(metrics_results),
           'ent_type': deepcopy(metrics_results),
           'partial':  deepcopy(metrics_results),
           'exact':    deepcopy(metrics_results)
          }


# results aggregated by entity type
evaluation_agg_entities_type = {e: deepcopy(results) for e in categories}

ki = 0
for true_ents0, pred_ents0 in zip(y_true, y_pred):
    
    true_ents = true_ents0.tolist()
    pred_ents = pred_ents0.tolist()
    
    try:
        while True:
            true_ents.remove('[CLS]')
    except ValueError:
        pass
    
    try:
        while True:
            true_ents.remove('[SEP]')
    except ValueError:
        pass
    
    try:
        while True:
            true_ents.remove('[PAD]')
    except ValueError:
        pass
    
    try:
        while True:
            pred_ents.remove('[CLS]')
    except ValueError:
        pass
    
    try:
        while True:
            pred_ents.remove('[SEP]')
    except ValueError:
        pass
    
    try:
        while True:
            pred_ents.remove('[PAD]')
    except ValueError:
        pass
    
    
    # compute results for one message
    tmp_results, tmp_agg_results = compute_metrics(
        collect_named_entities(true_ents), collect_named_entities(pred_ents), categories
    )
    
    #print(tmp_results)

    # aggregate overall results
    for eval_schema in results.keys():
        for metric in metrics_results.keys():
            results[eval_schema][metric] += tmp_results[eval_schema][metric]
            
    # Calculate global precision and recall
        
    results = compute_precision_recall_wrapper(results)


    # aggregate results by entity type
 
    for e_type in categories:

        for eval_schema in tmp_agg_results[e_type]:

            for metric in tmp_agg_results[e_type][eval_schema]:
                
                evaluation_agg_entities_type[e_type][eval_schema][metric] += tmp_agg_results[e_type][eval_schema][metric]
                
        # Calculate precision recall at the individual entity level
                
        evaluation_agg_entities_type[e_type] = compute_precision_recall_wrapper(evaluation_agg_entities_type[e_type])

In [ ]:
results

In [ ]:
evaluation_agg_entities_type

In [ ]:
evaluator = Evaluator(y_true, y_pred, categories)

In [ ]:
results2, results_agg = evaluator.evaluate()

In [ ]:
results2

In [ ]:
results_agg